In [1]:
import tensorflow as tf
import os
import keras
from keras import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib as mpl
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import UpSampling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ThresholdedReLU
from skimage.io import imsave, imread
import matplotlib.pyplot as plt
import numpy as np
import cv2
from keras.optimizers import Adam
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras.callbacks import EarlyStopping


Using TensorFlow backend.


In [2]:
resize_h = 224
resize_w = 224
np.random.seed(7)
data_dir = '/home/harry/viii sem/AI'

input_dir = data_dir + '/melanoma'
other_dir = data_dir + '/others'

train_dir = data_dir + '/gt'

In [3]:
def load_data():
    total=len(os.listdir(input_dir))+len(os.listdir(other_dir))
    print('Total number of images = ',total)
    imgs=[]
    y=[]
    i = 0

    images = os.listdir(input_dir)
    for image_name in images:
        img = cv2.imread(os.path.join(input_dir, image_name))
        img = cv2.resize(img, (resize_h, resize_w))
        img = img_to_array(img)
        imgs.append(img)
        y.append(1)
        #print('Done: {0}/{1} images'.format(i, total))
        i += 1
    len1 = i

    images = os.listdir(other_dir)
    for image_name in images:
        img = cv2.imread(os.path.join(other_dir, image_name))
        img = cv2.resize(img, (resize_h, resize_w))
        img = img_to_array(img)
        
        imgs.append(img)
        y.append(0)

        #print('Done: {0}/{1} images'.format(i, total))
        i += 1
    len0 = i - len1
    imgs = np.array(imgs, dtype="float")/255.0
    print('Loading done...')
    return imgs, y, len0, len1

In [4]:
X, Y, len0, len1 = load_data()
np.save( 'imgs_class.npy', X)
np.save('imgs_label.npy', Y)

Total number of images =  2000
Loading done...


In [5]:
#X = np.load('imgs_class.npy')
#Y = np.load('imgs_label.npy')

(trainX, testX, trainY, testY) = train_test_split(X,Y, test_size=0.25, random_state=42, shuffle = True)
del X
del Y
trainY = np_utils.to_categorical(trainY)
test2Y = testY
testY = np_utils.to_categorical(testY)

In [7]:
from keras.layers import *
from keras.models import Model
model = Sequential()
def get_simple_cnn_model():

    input_img = Input(shape=((resize_w,resize_h,3)))  

    x = Conv2D(6, (5, 5), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same',strides = (2,2))(x)
    x = Conv2D(6, (5, 5), activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2), padding='same',strides = (2,2))(x)
    x = Flatten()(x)
    x = Dense(120, kernel_initializer='normal', activation='relu')(x)
    x = Dense(84, kernel_initializer='normal', activation='relu')(x)
    x = Dense(2, kernel_initializer='normal', activation='softmax')(x)

    model = Model(input_img, x)   
    model.compile(optimizer='Adadelta', loss='categorical_crossentropy',metrics = ['accuracy'])

    print (model.summary())
    return model
	
model = get_simple_cnn_model()
print(' Weight = ', len0/len1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 6)       456       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 6)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 6)       906       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 6)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18816)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               2258040   
__________

In [8]:
classifier = model.fit(trainX,trainY, batch_size = 20, epochs = 30, verbose = True, validation_data = (testX, testY),class_weight = {0:1, 1:len0/len1})
PredY = model.predict(testX,verbose="True")
y_pred = []
for item in PredY:
	if item[0] > 0.5:
		y_pred.append(0)
	else:
		y_pred.append(1)

y_pred = np.array(y_pred,dtype = "int32")

Train on 1500 samples, validate on 500 samples
Epoch 1/30
1500/1500 [==============================] - 124s 82ms/step - loss: 1.1292 - acc: 0.6007 - val_loss: 0.6420 - val_acc: 0.8060
Epoch 2/30
1500/1500 [==============================] - 122s 81ms/step - loss: 1.1403 - acc: 0.6327 - val_loss: 0.5997 - val_acc: 0.8020
Epoch 3/30
1500/1500 [==============================] - 153s 102ms/step - loss: 1.1199 - acc: 0.5153 - val_loss: 0.6384 - val_acc: 0.6660
Epoch 4/30
1500/1500 [==============================] - 137s 91ms/step - loss: 1.0897 - acc: 0.5580 - val_loss: 0.5700 - val_acc: 0.8040
Epoch 5/30
1500/1500 [==============================] - 127s 85ms/step - loss: 1.0961 - acc: 0.5460 - val_loss: 0.7185 - val_acc: 0.3280
Epoch 6/30
1500/1500 [==============================] - 137s 92ms/step - loss: 1.0760 - acc: 0.5427 - val_loss: 0.4948 - val_acc: 0.8080
Epoch 7/30
1500/1500 [==============================] - 122s 81ms/step - loss: 1.0560 - acc: 0.6013 - val_loss: 0.5942 - val_acc: 

In [9]:
print('F1 =', f1_score(test2Y, y_pred, average=None)[0])

F1 = [0.87367178]


In [10]:
tn, fp, fn, tp = confusion_matrix(test2Y, y_pred).ravel()
matrix = [[tp, fn], [fp, tn]]
print('Confusion matrix = ', matrix)
print('precision_score: ', precision_score(test2Y, y_pred, pos_label=1, average='binary'))
print('recall_score: ', recall_score(test2Y, y_pred, pos_label=1, average='binary'))

Confusion matrix =  [[23, 73], [34, 370]]
precision_score:  0.40350877192982454
recall_score:  0.23958333333333334
